<a href="https://colab.research.google.com/github/ua-deti-information-retrieval/Neural-IR-hands-on/blob/main/RI_practical_tutorial_5_training.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install torch transformers ranx accelerate
!git clone https://github.com/ua-deti-information-retrieval/Neural-IR-hands-on.git



     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 99.2/99.2 kB 1.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 265.7/265.7 kB 5.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 318.0/318.0 kB 18.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 138.7/138.7 kB 18.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 24.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 228.3/228.3 kB 31.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.7/1.7 MB 90.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.6/1.6 MB 34.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 41.2/41.2 kB 5.2 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Installing backend dependencies ... done
  Preparing metadata (pyproject.toml) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 111.8/1

In [2]:
!wget "https://uapt33090-my.sharepoint.com/:u:/g/personal/aleixomatos_ua_pt/EVCkcYaBlo9GhK0pbIS3kbwBbni5AodWh8Br9q1sg4oWKA?e=nROIqj&download=1" -O dev_set_nq_bm25.jsonl
!wget "https://uapt33090-my.sharepoint.com/:u:/g/personal/aleixomatos_ua_pt/EeG_M5lDSNdKonIxIe7Op_8B-bfkupPZoxk7dRICJ8pS4Q?e=ffBdWy&download=1" -O dev_set_nq_gs.jsonl
!wget "https://uapt33090-my.sharepoint.com/:u:/g/personal/aleixomatos_ua_pt/EYkZF-SVwQlEo-2JShc5cvMBSDB7osa3jM_-hrR86_XJnA?e=hTYh3W&download=1" -O train_set_nq_neg.jsonl
!wget "https://uapt33090-my.sharepoint.com/:u:/g/personal/aleixomatos_ua_pt/EVNRdNHufJJCklmhU2VY3zgBkWsf4fO2mbamfGf2qZ3GRw?e=wfe3fj&download=1" -O train_set_nq_pos.jsonl
!wget "https://uapt33090-my.sharepoint.com/:u:/g/personal/aleixomatos_ua_pt/EY2sTtQg_QlHjkhYeE2fmFUBN4KJnmw2DSdJZGorEbMQhA?e=JqmpH2&download=1" -O collection_with_texts.jsonl

--2023-12-07 16:16:18--  https://uapt33090-my.sharepoint.com/:u:/g/personal/aleixomatos_ua_pt/EVCkcYaBlo9GhK0pbIS3kbwBbni5AodWh8Br9q1sg4oWKA?e=nROIqj&download=1
Resolving uapt33090-my.sharepoint.com (uapt33090-my.sharepoint.com)... 13.107.136.10, 13.107.138.10, 2620:1ec:8f8::10, ...
Connecting to uapt33090-my.sharepoint.com (uapt33090-my.sharepoint.com)|13.107.136.10|:443... connected.
HTTP request sent, awaiting response... 302 Found
Location: /personal/aleixomatos_ua_pt/Documents/@UA/Aulas/RI/2023/Dossier/Data/dataset_for_demo/dev_set_nq_bm25.jsonl?ga=1 [following]
--2023-12-07 16:16:19--  https://uapt33090-my.sharepoint.com/personal/aleixomatos_ua_pt/Documents/@UA/Aulas/RI/2023/Dossier/Data/dataset_for_demo/dev_set_nq_bm25.jsonl?ga=1
Reusing existing connection to uapt33090-my.sharepoint.com:443.
HTTP request sent, awaiting response... 200 OK
Length: 5022700 (4.8M) [application/octet-stream]
Saving to: ‘dev_set_nq_bm25.jsonl’

dev_set_nq_bm25.jso 100%[===================>]   4.79M  

# Starting here

In [3]:
!cd Neural-IR-hands-on && git pull

!cp Neural-IR-hands-on/trainer/* .

Already up to date.


In [5]:
from data import get_qrels, InferenceRankingIterator,  BioASQPointwiseIterator, InferenceDataset, create_training_dataset
from sampler import BasicSampler
from transformers import AutoTokenizer

model_checkpoint = "bert-base-uncased"

tokenizer = AutoTokenizer.from_pretrained(model_checkpoint)
tokenizer.model_max_length = 512

# torch.utils.data.Dataset (Iterable)
train_ds = create_training_dataset("train_set_nq_pos.jsonl", # "body"/"question" q_id:[doc_id] 60K
                                   "train_set_nq_neg.jsonl", # q_id:[{"id":doc_id, "score": doc_id}]
                                   "collection_with_texts.jsonl", # doc_id: text (title + " " + abstract)
                                    tokenizer=tokenizer,
                                    iterator_class=BioASQPointwiseIterator[BasicSampler],
                                    max_questions=500, # debug
                                            )
# torch.utils.data.Dataset (Iterable)
dev_ds = InferenceDataset("dev_set_nq_bm25.jsonl", #q_id:[doc_id ->>] BM25 top-1000 top-100 # 100 question -> 1000 docs
                          train_ds.collection,
                          tokenizer,
                          max_questions=10,
                          at=100, #max docs
                          gs_path="dev_set_nq_gs.jsonl", # q_id: [doc_id]
                          iterator_class=InferenceRankingIterator)


tokenizer_config.json:   0%|          | 0.00/28.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/570 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

In [7]:
from transformers import AutoModelForSequenceClassification

id2label = {0: "IRRELEVANT", 1: "RELEVANT"}
label2id = {"IRRELEVANT": 0, "RELEVANT": 1}

model = AutoModelForSequenceClassification.from_pretrained(
    model_checkpoint, num_labels=2, id2label=id2label, label2id=label2id
).to("cuda")

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at bert-base-uncased and are newly initialized: ['classifier.weight', 'classifier.bias']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [9]:
_iter = iter(train_ds)

print(next(_iter))
print(next(_iter))



{'input_ids': [101, 2054, 2024, 1996, 3937, 2916, 1997, 1996, 2111, 2104, 1996, 1057, 1012, 1055, 1012, 4552, 1029, 102, 2142, 2163, 4552, 2329, 2576, 9667, 2198, 18343, 2206, 1996, 14013, 4329, 1006, 24082, 1007, 2001, 1037, 2350, 3747, 9186, 2006, 1996, 3206, 3399, 1997, 2231, 3935, 2011, 2726, 7570, 19473, 2015, 1012, 18343, 3935, 1996, 6958, 1997, 9619, 1997, 1996, 9950, 1999, 2010, 2048, 15326, 2015, 1997, 2231, 1012, 2231, 1005, 1055, 4611, 2104, 1037, 2591, 3206, 2426, 1996, 11074, 2111, 2001, 2000, 3710, 1996, 2111, 2011, 8650, 2037, 2916, 1012, 2122, 3937, 2916, 2020, 2166, 1010, 7044, 1998, 3200, 1012, 102], 'token_type_ids': [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1], 'attention_mask': [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,

In [10]:
from transformers import TrainingArguments

training_args = TrainingArguments(num_train_epochs=5,
                                  learning_rate=2e-5, # AdamW
                                  weight_decay=0.01,
                                  per_device_train_batch_size=8,
                                  per_device_eval_batch_size=32,
                                  #evaluation_strategy= "epoch",
                                  dataloader_pin_memory=True,
                                  output_dir="trained_model",
                                  logging_strategy="steps",
                                  logging_first_step=True,
                                  logging_steps=100,
                                  save_strategy="epoch",
                                  save_total_limit=2,
                                  seed=42)

In [11]:
from ranker_trainer import RankerTrainer
from transformers import DataCollatorWithPadding
from collator import RankingCollator
from metrics import RanxMetrics

import torch
trainer = RankerTrainer(
      model=model,
      args=training_args,
      train_dataset=train_ds,
      #eval_dataset=dev_ds, # validation set
      tokenizer=tokenizer,
      data_collator=DataCollatorWithPadding(tokenizer=tokenizer), # apply to train
      #eval_data_collator=RankingCollator(tokenizer=tokenizer), # apply to val
      preprocess_logits_for_metrics=lambda logits, labels: torch.nn.functional.softmax(logits, dim=-1)[:,1], # prob for class 1 (relevant class)
      #compute_metrics=RanxMetrics(dev_ds.get_qrels()),
  )

/usr/local/lib/python3.10/dist-packages/transformers/deepspeed.py:23: FutureWarning: transformers.deepspeed module is deprecated and will be removed in a future version. Please import deepspeed modules directly from transformers.integrations
  warnings.warn(


In [12]:
trainer.train()

You're using a BertTokenizerFast tokenizer. Please note that with a fast tokenizer, using the `__call__` method is faster than using a method to encode the text followed by a call to the `pad` method to get a padded encoding.


Epoch,Training Loss,Validation Loss,Ndcg@100,Ndcg@10,Recall@10,Map@10
1,0.349700,No log,0.259716,0.238685,0.400000,0.186667
2,0.167800,No log,0.331161,0.306161,0.400000,0.275000


Token indices sequence length is longer than the specified maximum sequence length for this model (932 > 512). Running this sequence through the model will result in indexing errors
/usr/local/lib/python3.10/dist-packages/ranx/metrics/ndcg.py:72: NumbaTypeSafetyWarning: unsafe cast from uint64 to int64. Precision may be lost.
  scores[i] = _ndcg(qrels[i], run[i], k, rel_lvl, jarvelin)


KeyboardInterrupt: ignored

# Use a trained model to run inference!

## TODO reboot the machine

In [2]:
from transformers import AutoTokenizer, AutoModelForSequenceClassification

model_checkpoint = "trained_model/checkpoint-250"

tokenizer = AutoTokenizer.from_pretrained(model_checkpoint)
tokenizer.model_max_length = 512

model = AutoModelForSequenceClassification.from_pretrained(model_checkpoint).to("cuda")

In [3]:
from utils import load_collection_lookup
from data import InferenceRankingIterator, InferenceDataset
from collator import RankingCollator
import torch
from tqdm import tqdm
# i am using the dev dataset bc I do not have eval dataset for this example

# load the collection lookup
collection = load_collection_lookup("collection_with_texts.jsonl")

eval_ds = InferenceDataset("dev_set_nq_bm25.jsonl",
                            collection,
                            tokenizer,
                            at=100, #max docs pq q
                            iterator_class=InferenceRankingIterator)

dataloader = torch.utils.data.DataLoader(eval_ds,
                                         batch_size=32,
                                         pin_memory=True,
                                         collate_fn=RankingCollator(tokenizer))

In [11]:
_sample = next(iter(dataloader))

_sample["inputs"].input_ids.shape

torch.Size([32, 211])

In [12]:
from collections import defaultdict

run_dict = defaultdict(list) # q_id:[{"doc_id": doc_id, "score": prob}]

for sample in tqdm(dataloader):
  _inputs = sample["inputs"].to("cuda") # send inputs to gpu

  # run inference
  with torch.no_grad():
    logits = model(**_inputs).logits # 0->value, 1->value

    # based on the logits, we normalize to have a probabilistic distribution
    # and we extract the probability of that query-doc pair being relevant!
    prob_relevant_pair = torch.nn.functional.softmax(logits, dim=-1)[:,1] # [0-1]
    # 32,

  for i, q_id in enumerate(sample["id"]):
    run_dict[q_id].append({"doc_id":sample["doc_id"][i],
                           "score":prob_relevant_pair[i].item()})


100%|██████████| 313/313 [03:23<00:00,  1.54it/s]


In [ ]:
# lest sort by the neural IR model prob and we have our reranked order!
for q_id in run_dict:
  run_dict[q_id].sort(key=lambda x:-x["score"]) #prob


In [ ]:
run_dict["test100"]

[{'doc_id': 'doc3254', 'score': 0.9980644583702087},
 {'doc_id': 'doc3256', 'score': 0.9975541234016418},
 {'doc_id': 'doc3219', 'score': 0.9973419308662415},
 {'doc_id': 'doc1054415', 'score': 0.9953550100326538},
 {'doc_id': 'doc766388', 'score': 0.9945961833000183},
 {'doc_id': 'doc1176905', 'score': 0.9935491681098938},
 {'doc_id': 'doc2454961', 'score': 0.9931110143661499},
 {'doc_id': 'doc3259', 'score': 0.9913620352745056},
 {'doc_id': 'doc3230', 'score': 0.9826739430427551},
 {'doc_id': 'doc3250', 'score': 0.9778955578804016},
 {'doc_id': 'doc720706', 'score': 0.9538587927818298},
 {'doc_id': 'doc3243', 'score': 0.030262166634202003},
 {'doc_id': 'doc3232', 'score': 0.028278740122914314},
 {'doc_id': 'doc3267', 'score': 0.024174412712454796},
 {'doc_id': 'doc3229', 'score': 0.01920473948121071},
 {'doc_id': 'doc3263', 'score': 0.01466214656829834},
 {'doc_id': 'doc3265', 'score': 0.010578076355159283},
 {'doc_id': 'doc851273', 'score': 0.008489441126585007},
 {'doc_id': 'doc322